In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# Features engieneering

In [ ]:
users = pd.read_csv('../data/train_users_2.csv')
test_users = pd.read_csv('../data/test_users.csv')
sessions = pd.read_csv('../data/sessions.csv')
ages = pd.read_csv('../data/age_gender_bkts.csv')
countries = pd.read_csv('../data/countries.csv')
user_ids = set(sessions.user_id.values).intersection(users.id.values)

In [ ]:
from datetime import datetime

def days_between(d1, d2, t1="%Y-%m-%d", t2="%Y-%m-%d"):
    d1 = datetime.strptime(d1, t1)
    d2 = datetime.strptime(d2, t2)
    return abs((d2 - d1).days)

In [ ]:
users['month_account_created'] = users.apply(lambda r: int(r['date_account_created'].split('-')[1]), axis=1)

In [ ]:
users['target'] = users.apply(lambda r: 1 if r['country_destination'] != 'NDF' else 0, axis=1)

In [ ]:
data = pd.concat([users[users['target'] == 1].head(88000), users[users['target'] == 0].head(88000)])
data

In [ ]:
data['null_age'] = data['age'].isnull().astype(int)
data['age_correct'] = ((0 < data['age']) & (data['age'] < 100)).astype(int)
data['age_100'] = ((100 < data['age']) & (data['age'] < 2000)).astype(int)

In [ ]:
data['gender_unknown'] = (data['gender'] == '-unknown-').astype(int)

In [ ]:
data['aff_channel_api'] = (data['affiliate_channel'] == 'api').astype(int)
data['aff_channel_content'] = (data['affiliate_channel'] == 'content').astype(int)
data['aff_channel_other'] = (data['affiliate_channel'] == 'other').astype(int)

In [ ]:
data['first_dev_type_mac'] = (data['first_device_type'] == 'Mac').astype(int)
data['first_dev_type_unknown'] = (data['first_device_type'] == 'Other/Unknown').astype(int)

In [ ]:
data['created_account_spring'] = ((3 <= data['month_account_created']) & (data['month_account_created'] <= 6)).astype(int)
data['created_account_cold'] = ((9 <= data['month_account_created']) | (data['month_account_created'] <= 1)).astype(int)
data['created_account_summer'] = ((7 <= data['month_account_created']) & (data['month_account_created'] <= 8)).astype(int)

In [ ]:
data

In [ ]:
data = data.sample(frac=1)

In [ ]:
n = int(data.shape[0] * 0.7)
matrix_data = data.loc[:, 'null_age':].values

X_train = matrix_data[:n, :]
y_train = data['target'].values[:n]

X_test = matrix_data[n:, :]
y_test = data['target'].values[n:]

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=100)
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=99, max_depth=20)
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
model.feature_importances_

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100)
model.fit(X_train, y_train)
model.score(X_test, y_test)